# Assignment 2: Sqooping Data Into Spark

sqoop import --connect 'jdbc:sqlserver://bigdata220w18.database.windows.net:1433;database=week3' --driver 'com.microsoft.sqlserver.jdbc.SQLServerDriver' --connection-manager 'org.apache.sqoop.manager.SQLServerManager' --username 'bigdata' --password '5Zgv\6;8rM' -target-dir '/data/rfm/test4' --split-by 'HH_DEMOGRAPHIC.HOUSEHOLD_KEY' --query 'SELECT HH_DEMOGRAPHIC.HOUSEHOLD_KEY, HH_DEMOGRAPHIC.HOMEOWNER_DESC, TRANSACTION_DATA.day, TRANSACTION_DATA.basket_id, TRANSACTION_DATA.sales_value, TRANSACTION_DATA.QUANTITY, PRODUCT.PRODUCT_ID FROM HH_DEMOGRAPHIC, TRANSACTION_DATA, PRODUCT WHERE HH_DEMOGRAPHIC.HOUSEHOLD_KEY = TRANSACTION_DATA.HOUSEHOLD_KEY AND TRANSACTION_DATA.PRODUCT_ID = PRODUCT.PRODUCT_ID AND $CONDITIONS' --as-parquetfile

In [1]:
//import stuff for tests
import org.apache.spark.sql.SparkSession
val ss = SparkSession.
builder().
master("local").
appName("Spark in Motion Example").
config("spark.config.option", "some-value").
enableHiveSupport().
getOrCreate()

import ss.implicits._
import org.apache.spark.sql.functions._
import java.util.Date;

In [2]:
val rfm_raw = spark.read.parquet("hdfs:///data/rfm/test4/*.parquet")

In [3]:
rfm_raw.createOrReplaceTempView("rfm_temp")

In [4]:
val rfm_rpt = spark.sql("SELECT HOUSEHOLD_KEY, MIN(day) as recency, COUNT(DISTINCT basket_id) as frequency, SUM(sales_value) as monetary  FROM rfm_temp GROUP BY HOUSEHOLD_KEY ORDER BY recency, frequency, monetary")

In [5]:
val rfm_rpt_home = spark.sql("SELECT HOUSEHOLD_KEY, MIN(day) as recency, COUNT(DISTINCT basket_id) as frequency, SUM(sales_value) as monetary  FROM rfm_temp WHERE HOMEOWNER_DESC = 'Homeowner' GROUP BY HOUSEHOLD_KEY ORDER BY recency, frequency, monetary")

In [7]:
rfm_rpt.repartition(1).write.csv("hdfs:///data/rfm/results/rfm_all_report.csv")
rfm_rpt_home.repartition(1).write.csv("hdfs:///data/rfm/results/rfm_home_report.csv")